<a href="https://colab.research.google.com/github/woodRock/fishy-business/blob/main/code/gp/notebooks/Multi_tree_Genetic_Program_Part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-tree Genetic Program 

In [1]:
!pip install deap
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install skfeature-chappers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 KB 4.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 27 not upgraded.
Need to get 2,148 kB of archives.
After this operation, 7,427 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libgtk2.0-common all 

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_path_gdrive = '/content/drive/MyDrive/AI/Data'
dataset = 'Part' #@param ["Fish", "Part"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

In [3]:
"""
Data - data.py
==============

This is the data module. It contains the functions for loading, preparing, normalizing and encoding the data.
"""

import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import scipy.io


def encode_labels(y, y_test=None):
    """
    Convert text labels to numbers.

    Args:
        y: The labels.
        y_test: The test labels. Defaults to None.
    """
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    if y_test is not None:
        y_test = le.transform(y_test)
    return y, y_test, le


def load(filename, folder=''):
    """
    Load the data from the mat file.

    Args:
        filename: The name of the mat file.
        folder: The folder where the mat file is located.
    """
    path = folder + "/" + filename
    mat = scipy.io.loadmat(path)
    return mat


def prepare(mat):
    """
    Load the data from matlab format into memory. 

    Args:
        mat: The data in matlab format.
    """
    X = mat['X']   
    X = X.astype(float)
    y = mat['Y']    
    y = y[:, 0]
    return X,y


def normalize(X_train, X_test):
    """
    Normalize the input features within range [0,1].

    Args:
        X_train: The training data.
        X_test: The test data.
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test


file = load(f'{dataset}.mat', folder=data_path_gdrive)
X,y = prepare(file)
X,_ = normalize(X,X)
y, _, le = encode_labels(y)
labels = le.inverse_transform(np.unique(y))
classes, class_counts = np.unique(y, return_counts=True)
n_features = X.shape[1]
n_instances = X.shape[0]
n_classes = len(classes)
class_ratios = np.array(class_counts) / n_instances

print(f"Class Counts: {class_counts}, Class Ratios: {class_ratios}")
print(f"Number of features: {n_features}\nNumber of instances: {n_instances}\nNumber of classes {n_classes}.")

Class Counts: [24 34 21 34 21 20], Class Ratios: [0.15584416 0.22077922 0.13636364 0.22077922 0.13636364 0.12987013]
Number of features: 4800
Number of instances: 154
Number of classes 6.


## Activation Function

## Operators

In [4]:
import math
import copy
import random
import operator
from re import I
from operator import attrgetter
from functools import wraps, partial

import numpy as np

from deap import algorithms
from deap.algorithms import varAnd
from deap import base, creator, tools, gp
from deap.gp import PrimitiveTree, Primitive, Terminal

from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold

pset = gp.PrimitiveSet("MAIN", n_features)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
# pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

## Fitness Function

In [5]:
toolbox = base.Toolbox()

minimized = False
if minimized: 
    weight = -1.0 
else: 
    weight = 1.0

weights = (weight,) 

if minimized: 
    creator.create("FitnessMin", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMin)
else:
    creator.create("FitnessMax", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMax)

def quick_evaluate(expr: PrimitiveTree, pset, data, prefix='ARG'):
    result = None
    stack = []
    for node in expr:
        stack.append((node, []))
        while len(stack[-1][1]) == stack[-1][0].arity:
            prim, args = stack.pop()
            if isinstance(prim, Primitive):
                result = pset.context[prim.name](*args)
            elif isinstance(prim, Terminal):
                if prefix in prim.name:
                    result = data[:, int(prim.name.replace(prefix, ''))]
                else:
                    result = prim.value
            else:
                raise Exception
            if len(stack) == 0:
                break # If stack is empty, all nodes should have been seen
            stack[-1][1].append(result)
    return result

def compileMultiTree(expr, pset, data=X):
    """Compile the expression represented by a list of trees. 

    A variation of the gp.compileADF method, that handles Multi-tree GP. 

    Args: 
        expr: Expression to compile. It can either be a PrimitiveTree,
                 a string of Python code or any object that when
                 converted into string produced a valid Python code
                 expression.
        pset: Primitive Set

    Returns: 
        A set of functions that correspond for each tree in the Multi-tree. 
    """
    funcs = []
    gp_tree = None
    func = None

    for subexpr in expr:
        gp_tree = gp.PrimitiveTree(subexpr)
        # 5x speedup by manually parsing GP tree (Zhang 2022) https://mail.google.com/mail/u/0/#inbox/FMfcgzGqQmQthcqPCCNmstgLZlKGXvbc
        func = quick_evaluate(gp_tree, pset, data, prefix='ARG')
        funcs.append(func)

    # Hengzhe's method returns the features in the wrong rotation for multi-tree
    features = np.array(funcs).T 
    return features

# Class dependent subtrees, one for each class, winner takes all. 

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.expr, n=n_classes)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", compileMultiTree)

In [6]:
def xmate(ind1, ind2):
    """ Reproduction operator for multi-tree GP, where trees are represented as a list.

    Crossover happens to a subtree that is selected at random. 
    Crossover operations are limited to parents from the same tree.

    FIXME: Have to compile the trees (manually), which is frustrating. 
    
    Args: 
        ind1 (Individual): The first parent. 
        ind2 (Individual): The second parent 

    Returns:
        ind1, ind2 (Individual, Individual): The children from the parents reproduction. 
    """
    n = range(len(ind1))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1, g2 = gp.PrimitiveTree(ind1[tree_idx]), gp.PrimitiveTree(ind2[tree_idx])
        if tree_idx == selected_tree_idx:
            ind1[tree_idx], ind2[tree_idx] = gp.cxOnePoint(g1, g2)
        else: 
            ind1[tree_idx], ind2[tree_idx] = g1, g2
    return ind1, ind2


def xmut(ind, expr):
    """ Mutation operator for multi-tree GP, where trees are represented as a list. 

    Mutation happens to a tree selected at random, when an individual is selected for crossover. 

    FIXME: Have to compile the trees (manually), which is frustrating. 

    Args: 
        ind: The individual, a list of GP trees. 
    """
    n = range(len(ind))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1 = gp.PrimitiveTree(ind[tree_idx])
        if tree_idx == selected_tree_idx:
            indx = gp.mutUniform(g1, expr, pset)
            ind[tree_idx] = indx[0]
        else:
            ind[tree_idx] = g1
    return ind,


def evaluate_classification(individual, X=X, y=y, verbose=False):    
    """ Evaluate balanced classification accuracy over stratified k-fold cross validation. """ 
    def predict(X_idx, y_t):
        """ Winner-takes-all predictions with GP-tree for a dataset."""
        y_preds = []   
        for tree_output in funcs[X_idx]:
            winner_takes_all = np.argmax(tree_output)
            y_preds.append(winner_takes_all)
        accuracy = balanced_accuracy_score(y_t, y_preds)
        return accuracy

    def cross_validate(X,y,k=10):
        """ Performs k-fold cross validation to evaluate a dataset. """
        train_accs = []
        test_accs = [] 
        skf = StratifiedKFold(n_splits=k)
        for train_idx, test_idx in skf.split(X,y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            train_acc = predict(train_idx, y_train)
            train_accs.append(train_acc)
            # 2x speedup in evaluation time.
            if verbose:
                test_acc = predict(test_idx, y_test)
                test_accs.append(test_acc)
        avg_train_acc = np.mean(train_accs) 
        avg_test_acc = np.mean(test_accs) if verbose else 0
        return avg_train_acc, avg_test_acc

    funcs = toolbox.compile(expr=individual, pset=pset)
    avg_train_acc, avg_test_acc = cross_validate(X,y)

    if verbose:
        print(f"Average train accuracy: {avg_train_acc}, Average test accuracy: {avg_test_acc}")
    
    return avg_train_acc, 


toolbox.register('evaluate', evaluate_classification)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", xmate)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", xmut, expr=toolbox.expr_mut)


def staticLimit(key, max_value):
    """
    A variation of gp.staticLimit that works for Multi-tree representation. 
    This works for our altered xmut and xmate genetic operators for mutli-tree GP. 
    If tree depth limit is exceeded, the genetic operator is reverted. 

    When an invalid (over the limit) child is generated,
    it is simply replaced by one of its parents, randomly selected.
    
    Args:
        key: The function to use in order the get the wanted value. For
             instance, on a GP tree, ``operator.attrgetter('height')`` may
             be used to set a depth limit, and ``len`` to set a size limit.
        max_value: The maximum value allowed for the given measurement. 
             Defaults to 17, the suggested value in (Koza 1992)
    
    Returns: 
        A decorator that can be applied to a GP operator using \
        :func:`~deap.base.Toolbox.decorate`

    References:
        1. Koza, J. R. G. P. (1992). On the programming of computers by means 
            of natural selection. Genetic programming.
    """

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            keep_inds = [[copy.deepcopy(tree) for tree in ind] for ind in args]
            new_inds = list(func(*args, **kwargs))
            for ind_idx, ind in enumerate(new_inds):
                for tree_idx, tree in enumerate(ind):
                    if key(tree) > max_value:
                        random_parent = random.choice(keep_inds)
                        new_inds[ind_idx][tree_idx] = random_parent[tree_idx]
            return new_inds
        return wrapper
    return decorator

# See https://groups.google.com/g/deap-users/c/pWzR_q7mKJ0
toolbox.decorate("mate", staticLimit(key=operator.attrgetter("height"), max_value=8))
toolbox.decorate("mutate", staticLimit(key=operator.attrgetter("height"), max_value=8))


def SimpleGPWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """
    Elitism for Multi-Tree GP for Multi-Class classification. 
    A variation of the eaSimple method from the DEAP library that supports 

    Elitism ensures the best individuals (the elite) from each generation are 
    carried onto the next without alteration. This ensures the quality of the 
    best solution monotonically increases over time. 
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    
    if verbose:
        print(logbook.stream)

    for gen in range(1, ngen + 1):
        offspring = toolbox.select(population, len(population) - hof_size)
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        offspring.extend(halloffame.items)
        halloffame.update(offspring)
        population[:] = offspring
        
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        
        if verbose:
            print(logbook.stream)

    return population, logbook


def train(generations=100, population=100, elitism=0.1, crossover_rate=0.5, mutation_rate=0.1):
    """
    This is a Multi-tree GP with Elitism for Multi-class classification. 

    Args:
        generations: The number of generations to evolve the populaiton for. 
        elitism: The ratio of elites to be kept between generations. 
        crossover_rate: The probability of a crossover between two individuals. 
        mutation_rate: The probability of a random mutation within an individual.  

    Returns:
        pop: The final population the algorithm has evolved. 
        log: The logbook which can record important statistics. 
        hof: The hall of fame contains the best individual solutions.
    """
    random.seed(420)
    pop = toolbox.population(n=population)
    
    mu = round(elitism * population)
    if elitism > 0:
        # See https://www.programcreek.com/python/example/107757/deap.tools.HallOfFame
        hof = tools.HallOfFame(mu)
    else:
        hof = None
    
    # Fitness is a balanced accuracy + hamming distance regularization. 
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)

    # Size, is the tree with the maximum length in the multi-tree
    length = lambda a: np.max(list(map(len, a)))
    stats_size = tools.Statistics(length)

    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)

    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = SimpleGPWithElitism(pop, toolbox, crossover_rate, mutation_rate, 
                                   generations, stats=mstats, halloffame=hof, 
                                   verbose=True)
    return pop, log, hof


"""
DeJong (1975), p=50-100, m=0.001, c=0.6 
Grefenstette (1986), p=30, m=0.01, c=0.95
Schaffer et al., (1989), p=20-30, m=0.005-0.01, c=0.75-0.95

References:
    1. Patil, V. P., & Pawar, D. D. (2015). The optimal crossover or mutation 
    rates in genetic algorithm: a review. International Journal of Applied 
    Engineering and Technology, 5(3), 38-41.
"""

beta = 3
population = n_features * beta
generations = 300
elitism = 0.1
crossover_rate = 0.8
mutation_rate = 0.2

assert crossover_rate + mutation_rate == 1, "Crossover and mutation sums to 1 (to please the Gods!)"

pop, log, hof = train(generations, population, elitism, crossover_rate, mutation_rate)

   	      	                                 fitness                                 	                     size                     
   	      	-------------------------------------------------------------------------	----------------------------------------------
gen	nevals	avg     	gen	max     	min      	nevals	std      	avg    	gen	max	min	nevals	std   
0  	14400 	0.167814	0  	0.313243	0.0279211	14400 	0.0382098	5.67556	0  	7  	2  	14400 	1.6046
1  	10786 	0.199173	1  	0.343871	0.0502958	10786 	0.0339728	5.80896	1  	13 	2  	10786 	1.68365
2  	10894 	0.223335	2  	0.35094 	0.0637276	10894 	0.032591 	5.90604	2  	14 	2  	10894 	1.79234
3  	10879 	0.241619	3  	0.35094 	0.0703537	10879 	0.0324731	6.01806	3  	18 	2  	10879 	1.84638
4  	10874 	0.256582	4  	0.363549	0.0784097	10874 	0.0334379	6.17812	4  	15 	2  	10874 	1.88477
5  	10948 	0.269165	5  	0.391756	0.0808423	10948 	0.0347849	6.35986	5  	17 	2  	10948 	1.89592
6  	10864 	0.280955	6  	0.391756	0.0593369	10864 	0.0357225	6.50694	6  	1

In [7]:
# The invidiual with best fitness. 
best = hof[0]

# Classification accuracy for the unseen train set. 
evaluate_classification(best, verbose=True)

Average train accuracy: 0.8683920864396246, Average test accuracy: 0.8680555555555556


(0.8683920864396246,)

## Visualization

In [8]:
from deap import base, creator, gp
import pygraphviz as pgv

multi_tree = hof[0]
for t_idx,tree in enumerate(multi_tree): 
    nodes, edges, labels = gp.graph(tree)

    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(f"tree-{t_idx}.pdf")

## Changelog 

| Date | Title | Description | Update | 
| --- | --- | --- | ---- | 
| 2022-08-21 17:30 | Multi-Objective - Onehot Encoding | Change to multi-objective problem, one-vs-all with a tree classifier for each class.<br> Y labels are encoded in onehot encodings, error is absolute difference between $|\hat{y} - y|$| 
| 2022-08-22 20:44 | Non-linearity |  Introduce $ round . sigmoid $ to evaluate_classification() method.<br>Previously, we push each tree to predict either a 0 or 1 value with the onehot encoding representation.<br>Now, the non-linearity will map any negative value to a negative class 0, and any positive value to positive class 1.|
| 2022-08-22 21:06 | ~~Genetic operators for tree with worst fitness~~ | Only apply the genetic operators, crossover and mutation, to the tree with the worst fitness.<br> This guarantees monotonic improvement for the Multi-tree between generations, the best performing tree remain unaltered.| (Update) This was very slow, and inefficient,<br> basically turned the GP into a single objective,<br>that balances multi-objective fitness functions. | 
| 2022-08-22 21:15 | Halloffame Equality Operator | Numpy equality function (operators.eq) between two arrays returns the equality element wise,<br>which raises an exception in the if similar() check of the hall of fame. <br> Using a different equality function like numpy.array_equal or numpy.allclose solve this issue.| 
| 2022-08-22 23:22 | Elitism as aggregate best tree | Perform elitsim by constructing the best tree, as the tree with best fitness from each clas.<br>The goal is to have monotonous improvement across the multiple objective functions.| 
| 2022-08-22 23:32 | Update fitness for elite | The elitism was not working as intended, as the multi-objectives didn't appear to increase monotnously.<br> This was because the aggregate fitness was not being assigned to the best individual after it was created.<br>Therefore the best invidiual was not passed on to the next generation. | 
| 2022-08-22 02:28 | staticLimit max height | Rewrite the gp.staticLimit decorator function to handle the Multi-tree representation.<br>Note: size and depth are different values!<br>depth is the maximum length of a root-to-leaf traversal,<br>size is the total number of nodes.| 
| 2022-08-24 9:37 | Evaluate Mutli-tree train accuracy | Take the classification accuracy as the argmax for the aggregate multitree.<br> 74% training accuracy, which is not ideal, but this shall improve with time.| 
| 2022-08-25 13:30 | Single-objective fitness | Change the fitness function to a single objective fitness function.<br>This forces the multi-tree GP to find the best tree subset for one-vs-rest classification performance.| 
| 2022-08-25 20:01 | Fitness = Balanced accuracy + distance measure | Implement the fitness function for MCIFC, but for multi-class classification from (Tran 2019) |
| 2022-08-26 21:27 | Sklearn Balanced Accuracy | Changed to the balanced accuracy metric from sklearn.<br>This is much easier to use for now, probably faster than the previous method as well. | 
| 2022-09-05 17:00 | Reject invalid predictions | Change the fitness function to reject invalid predictioctions outright -<br>e.g. multi-label or zero-label predictions<br>- when computing the balanced accuracy for the fitness function. |
| 2022-09-26 | Winner-takes-all | Use a winner-takes-all approach for the prediction.<br>The class depedendent subtree with the largest value is the predicted class. | 
| 2022-10-14 20:23 | Hengzhe's 5x speedup | Implemented Henzhe's 5x speedup for GP tree compilation with DEAP library. | 
| 2022-10-14 20:23 | Crossvalidation | Implemented k-fold crossvalidation for assessing the fitness of an indvidual. | 
| 2022-10-15 02:03 | Train, Validation, Test | The test set is reserved for final comparison, fitness is based on crossvalidation performance on training set only.<br>This gives an accurate evaluation for performance on unseen data (without data leakage).| 